In [2]:
import sys
import os

# Workaround to make packages work in both Jupyter notebook and Python
MODULE_ROOT_NAME = "AgeEstimator"
MODULE_PATHS = [
    os.path.abspath(os.path.join('..')),
    os.path.abspath(os.path.join('../..')),
    os.path.abspath(os.path.join('../../..'))
]
MODULE_PATHS = list(
    filter(lambda x: x.endswith(MODULE_ROOT_NAME), MODULE_PATHS))
MODULE_PATH = MODULE_PATHS[0] if len(MODULE_PATHS) == 1 else ""
if MODULE_PATH not in sys.path:
    sys.path.append(MODULE_PATH)
    
from server.data.dataset import DataLoader
from server.models.cnn.model import IMAGE_SIZE, INPUT_SHAPE, CURR_DIR

Using TensorFlow backend.
/WAVE/apps/eb/software/SciPy-bundle/2019.03-fosscuda-2019a/lib/python3.7/site-packages/pandas/core/series.py:4141: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  infer_datetime_format=infer_datetime_format)


In [3]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from mtcnn.mtcnn import MTCNN
from keras_vggface.utils import preprocess_input
from keras_vggface.vggface import VGGFace
from multiprocessing import cpu_count
from threading import Thread
from PIL import Image
import queue

In [4]:
# Shared model to create bottleneck features

model = VGGFace(model="resnet50", include_top=False,
                   input_shape=INPUT_SHAPE)

# WORKERS = cpu_count()
WORKERS = 20
THREADS_PER_CORE = 2

In [5]:
def extract_face(filename, input_image=None, img_size=IMAGE_SIZE):
    r"""Extract a single face from a given photograph"""

    img = input_image if input_image else plt.imread(filename)
    
    # Create the detector, using default weights
    detector = MTCNN()
    detection = detector.detect_faces(img)

    # Extract the bounding box from the first face
    x1, y1, width, height = detection[0]['box']
    x2, y2 = x1 + width, y1 + height
    face = img[y1:y2, x1:x2]
    
    # Resize pixels to the model size
    image = Image.fromarray(face)
    image = image.resize(img_size)
    face_array = np.asarray(image)
    return face_array.astype('float32')

In [6]:
def preprocess_image(img):
    print("Processing image...")
    img = extract_face(input_image=img)
    img = np.expand_dims(img, axis=0)
    
    print("Preprocessing input...")
    processed = preprocess_input(img, version=2)
    
    print("Creating bottleneck features...")
    y_hat = model.predict(processed)
    
    print("Done.")
    return y_hat

In [11]:
def preprocess_dataset(idx=0, total_workers=1):
    dl = DataLoader()
    
    x_train, y_train = dl.load_train()
    x_test, y_test   = dl.load_test()
    
    train_limit = len(x_train) // total_workers
    test_limit  = len(x_test) // total_workers
    
    x_train = x_train[idx * train_limit : (idx + 1) * train_limit]
    y_train = y_train[idx * train_limit : (idx + 1) * train_limit]
    x_test  = x_test[idx * test_limit : (idx + 1) * test_limit]
    y_test  = y_test[idx * test_limit : (idx + 1) * test_limit]
    
    print("[Worker%d]: Processing training images..." % idx)
    x_train_labels   = []
    x_train_features = []
    
    for i, fname in enumerate(x_train):
        try:
            feature = extract_face(fname)
        except Exception as e:
            print("[Worker%d]: Exception: %s" % (idx, str(e)))
            continue
        x_train_labels.append(y_train[i])
        x_train_features.append(feature)
        
    print("[Worker%d]: Processing test images..." % idx)
    x_test_labels   = []
    x_test_features = []
    for i, fname in enumerate(x_test):
        try:
            feature = extract_face(fname)
        except Exception as e:
            print("[Worker%d]: Exception: %s" % (idx, str(e)))
            continue
        x_test_labels.append(y_test[i])
        x_test_features.append(feature)
    
    print("[Worker%d]: Preprocessing inputs..." % idx)
    x_train_processed = preprocess_input(x_train_features, version=2)
    x_test_processed  = preprocess_input(x_test_features, version=2)
    
    print("[Worker%d]: Creating bottleneck features..." % idx)
    p_train = model.predict(x_train_processed)
    p_test  = model.predict(x_test_processed)
    
    print("[Worker%d]: Saving features..." % idx)
    np.save(os.path.join(CURR_DIR, "features-train-%d" % idx), p_train)
    np.save(os.path.join(CURR_DIR, "features-test-%d" % idx), p_test)
    np.save(os.path.join(CURR_DIR, "labels-train-%d" % idx), x_train_labels)
    np.save(os.path.join(CURR_DIR, "labels-test-%d" % idx), x_test_labels)
    
    print("[Worker%d]: Done." % udx)
    return p_train, p_test, x_train_labels, x_test_labels, idx

In [8]:
def preprocess_dataset_parallel():
    q = queue.Queue()  # For putting and getting results of thread
    
    threads_count = WORKERS * THREADS_PER_CORE
    
    print("Using %d threads." % threads_count)
    
    do_preprocess = lambda idx, total, q: q.put(preprocess_dataset(idx, total))
    threads = [Thread(
        target=do_preprocess,
        args=(idx, threads_count, q)
    ) for idx in range(threads_count)]
    
    for t in threads: t.start()
    for t in threads: t.join()
        
    res = []
    for t in threads: res.append(q.get())
        
    return res

In [12]:
res = preprocess_dataset_parallel()